### myBERT   
Create and train my own BERT language model.  
Article: https://www.kdnuggets.com/2021/08/train-bert-model-scratch.html?utm_source=pocket_mylist    
Christoph Windheuser    
August 19, 2021

pip install datasets

In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset('oscar', 'unshuffled_deduplicated_it')


Dataset oscar downloaded and prepared to /home/christoph/.cache/huggingface/datasets/oscar/unshuffled_deduplicated_it/1.0.0/84838bd49d2295f62008383b05620571535451d84545037bb94d6f3501651df2. Subsequent calls will reuse this data.


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 28522082
    })
})

In [4]:
dataset['train']

Dataset({
    features: ['id', 'text'],
    num_rows: 28522082
})

In [5]:
dataset['train'].features

{'id': Value(dtype='int64', id=None), 'text': Value(dtype='string', id=None)}

In [8]:
dataset['train'][0]

{'id': 0,
 'text': "La estrazione numero 48 del 10 e LOTTO ogni 5 minuti e' avvenuta sabato 15 settembre 2018 alle ore 04:00 a Roma, nel Centro Elaborazione Dati della Lottomatica Italia (ora GTech SpA), con la supervisione della Amministrazione Autonoma dei Monopoli di Stato (AAMS), incaricata di vigilare sulla regolarità delle operazioni di sorteggio.\nIl Montepremi della 48ª estrazione viene ripartito tra i vincitori delle singole categorie di premio.\nRicorda di controllare il Numero ORO 53. E, se lo hai giocato, anche il DOPPIO ORO 53 e 66. Se indovini puoi vincere premi più ricchi.\nIl nostro sito web impiega cookies per migliorare la navigazione del visitatore. L’utente è consapevole che, continuando a visitare il nostro sito web, accetta l’utilizzo dei cookies Accetto Informazioni\n(C) Copyright 2013-2017 10elotto.biz | Il presente sito è da considerarsi un sito indipendente, NON collegato alla rete ufficiale Gtech SpA."}

In [13]:
from tqdm.auto import tqdm

text_data = []
file_count = 0

!pwd

for sample in tqdm(dataset['train']):
    sample = sample['text'].replace('\n', '')
    text_data.append(sample)
    if len(text_data) == 10_000:
        # once we git the 10K mark, save to file
        with open(f'data/text/oscar_it/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
# after saving in 10K chunks, we will have ~2082 leftover samples, we save those now too
with open(f'data/text/oscar_it/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))


/home/christoph/dev/myBERT


In [14]:
from pathlib import Path
paths = [str(x) for x in Path('data/text/oscar_it').glob('**/*.txt')]


In [16]:
paths[:5]

['data/text/oscar_it/text_1732.txt',
 'data/text/oscar_it/text_1927.txt',
 'data/text/oscar_it/text_2374.txt',
 'data/text/oscar_it/text_469.txt',
 'data/text/oscar_it/text_2340.txt']

In [17]:
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()


In [18]:
tokenizer.train(files=paths[:5], vocab_size=30_522, min_frequency=2,
                special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])


In [20]:
import os

os.mkdir('./filiberto')

tokenizer.save_model('filiberto')

['filiberto/vocab.json', 'filiberto/merges.txt']

In [21]:
from transformers import RobertaTokenizer

# initialize the tokenizer using the tokenizer we initialized and saved to file
tokenizer = RobertaTokenizer.from_pretrained('filiberto', max_len=512)


In [22]:
# test our tokenizer on a simple sentence
tokens = tokenizer('ciao, come va?')

In [23]:
print (tokens)

{'input_ids': [0, 15253, 16, 478, 589, 35, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}


In [24]:
tokens.input_ids

[0, 15253, 16, 478, 589, 35, 2]

### MLM Input Pipeline (MLM = Masked Language Modeling)

In [6]:
import torch

def mlm(tensor):
    rand = torch.rand(tensor.shape)
    mask_arr = (rand < 0.15) * (tensor > 2)
    for i in range(tensor.shape[0]):
        selection = torch.flatten(mask_arr[i].nonzero()).tolist()
        tensor[i, selection] = 4
    return tensor


In [31]:
paths = [str(x) for x in Path('data/text/oscar_it').glob('**/*.txt')]
paths[:5]

['data/text/oscar_it/text_1732.txt',
 'data/text/oscar_it/text_1927.txt',
 'data/text/oscar_it/text_2374.txt',
 'data/text/oscar_it/text_469.txt',
 'data/text/oscar_it/text_2340.txt']

In [40]:
input_ids = []
mask      = []
labels    = []

for path in tqdm(paths[:50]):
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n')
    sample = tokenizer(lines, max_length=512, padding='max_length', truncation=True)
    labels.append(sample.input_ids)
    mask.append(sample.attention_mask)
    tmp_tensor = torch.tensor(sample.input_ids).detach()
    input_ids.append(mlm(tmp_tensor))


In [42]:
# Save objects as pickle files
#
import pickle

f_input_ids = open("input_ids.pkl", "wb")
pickle.dump(input_ids, f_input_ids)
f_input_ids.close()

f_mask = open("mask.pkl", "wb")
pickle.dump(mask, f_mask)
f_mask.close()

f_labels = open("labels.pkl", "wb")
pickle.dump(labels, f_labels)
f_labels.close()


NameError: name 'f_label' is not defined

In [2]:
# Load objects back from the pickle files
# The notebook can be started at this point

import torch
import pickle

f_input_ids = open("input_ids.pkl", "rb")
input_ids   = pickle.load(f_input_ids)
f_input_ids.close()

f_mask = open("mask.pkl", "rb")
mask   = pickle.load(f_mask)
f_mask.close()

f_labels = open("labels.pkl", "rb")
labels   = pickle.load(f_labels)
f_labels.close()


In [8]:
input_ids[:2]

[tensor([[    0,    37,    42,  ...,     1,     1,     1],
         [    0,  2938,     4,  ...,     1,     1,     1],
         [    0, 28423,   290,  ...,     1,     1,     1],
         ...,
         [    0,  1414,  6342,  ...,  1227,  6133,     2],
         [    0,    37, 18027,  ..., 13766,    16,     2],
         [    0,  1414,  3790,  ...,     1,     1,     1]]),
 tensor([[    0,     4, 21935,  ...,     1,     1,     1],
         [    0,    54,   956,  ...,     1,     1,     1],
         [    0,    48,   324,  ...,   452,     4,     2],
         ...,
         [    0,  2314,    18,  ...,     1,     1,     1],
         [    0,     4,     4,  ...,     1,     1,     1],
         [    0,     4,  5261,  ...,     1,     1,     1]])]

In [15]:
 torch.cat(input_ids[:2])

TypeError: cat(): argument 'tensors' (position 1) must be tuple of Tensors, not Tensor

In [14]:
input_ids = torch.cat(input_ids)
mask      = torch.cat(torch.tensor(mask))
labels    = torch.cat(labels)


TypeError: cat(): argument 'tensors' (position 1) must be tuple of Tensors, not Tensor

In [12]:
input_ids[0][:10]

tensor([   0,   37,   42, 1664, 2753,   19,    4, 5217,    4,  551])

In [13]:
labels[0][:10]

[[0,
  37,
  42,
  1664,
  2753,
  19,
  3382,
  5217,
  27582,
  551,
  521,
  6725,
  526,
  343,
  299,
  2074,
  275,
  656,
  546,
  343,
  299,
  7702,
  303,
  12083,
  306,
  6973,
  509,
  19492,
  286,
  17688,
  551,
  10458,
  1367,
  3135,
  37,
  42,
  1664,
  2753,
  19,
  3382,
  5217,
  27582,
  551,
  521,
  6725,
  526,
  343,
  299,
  2074,
  275,
  656,
  546,
  343,
  299,
  7702,
  303,
  12083,
  306,
  6973,
  509,
  19492,
  286,
  17688,
  59,
  9955,
  1095,
  27739,
  403,
  7842,
  1367,
  3135,
  13,
  551,
  521,
  1410,
  5620,
  452,
  4919,
  1320,
  303,
  315,
  1030,
  3378,
  381,
  660,
  303,
  5447,
  488,
  9713,
  293,
  21159,
  374,
  6973,
  16,
  367,
  490,
  303,
  5579,
  579,
  2617,
  313,
  565,
  343,
  315,
  660,
  2074,
  313,
  684,
  303,
  767,
  19947,
  306,
  683,
  3137,
  910,
  1865,
  7094,
  313,
  7827,
  306,
  683,
  846,
  509,
  23850,
  286,
  17688,
  374,
  995,
  390,
  5965,
  290,
  376,
  15316,
  9673,
  